In [43]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_community import chains
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community import document_loaders

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [44]:
loader=PyPDFLoader('Job Description.pdf')
docs=loader.load()
docs

[Document(metadata={'producer': 'iText® Core 7.2.2 (AGPL version), pdfHTML 4.0.2 (AGPL version) ©2000-2022 iText Group NV', 'creator': 'PyPDF', 'creationdate': '2025-09-23T15:43:01+00:00', 'moddate': '2025-09-23T15:43:01+00:00', 'source': 'Job Description.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Associate System Engineer\nEngineering 0 year FullTime Indore\nCompany Description:\nAmerican Chase is a leading IT consulting company in central India that provides staffing services to \nFortune\n500 clients across the US. Established in 2016, American Chase has expanded its service offerings to \nprovide\nIT services and solutions to a host of industries including banking - finance, communication, consumer\nproducts, healthcare, insurance, technology, media-entertainment, publishing, education, healthcare, \nenergy\nand manufacturing industries. With offices located in Portland, Oregon, and Indore, (MP) India. American\nChase is committed to customer satisfaction 

In [45]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunk_documents=text_splitter.split_documents(docs)
chunk_documents

[Document(metadata={'producer': 'iText® Core 7.2.2 (AGPL version), pdfHTML 4.0.2 (AGPL version) ©2000-2022 iText Group NV', 'creator': 'PyPDF', 'creationdate': '2025-09-23T15:43:01+00:00', 'moddate': '2025-09-23T15:43:01+00:00', 'source': 'Job Description.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Associate System Engineer\nEngineering 0 year FullTime Indore\nCompany Description:\nAmerican Chase is a leading IT consulting company in central India that provides staffing services to \nFortune\n500 clients across the US. Established in 2016, American Chase has expanded its service offerings to \nprovide\nIT services and solutions to a host of industries including banking - finance, communication, consumer\nproducts, healthcare, insurance, technology, media-entertainment, publishing, education, healthcare, \nenergy\nand manufacturing industries. With offices located in Portland, Oregon, and Indore, (MP) India. American\nChase is committed to customer satisfaction 

In [46]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
db=FAISS.from_documents(chunk_documents,OllamaEmbeddings())
db

In [47]:
retriever=db.as_retriever(search_kwargs={"k":3})

In [48]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
 
# LLM
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = Ollama(model="llama2")
# llm = Ollama(model="mistral")
# Custom prompt template
prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant. Use the following context to answer the question.in a concise manner .
 
Context:
{context}
 
Question:
{question}
 
Answer concisely.
""")

In [49]:
from langchain_community.vectorstores import FAISS
def ask_question(query):
    # FAISS se context retrieve karo
    docs = retriever.invoke(query)
    context = "\n\n".join([d.page_content for d in docs])
   
    # Prompt format karo
    formatted_prompt = prompt.format(context=context, question=query)
   
    # LLM se response lo
    response = llm.invoke(formatted_prompt)
    return response

In [42]:
query = " how many total items are there "
answer = ask_question(query)
print(answer)

There are 10 items in the job description:

1. Batch required: Graduating class of 2025
2. Job profile/title: Associate System Engineer
3. Job type/shift: Full-time (night shift)
4. Eligible degree & branches: BE/B.Tech - All Branches, MCA - All Branches
5. Eligibility criteria: 55% throughout (10th, 12th, and graduation)
6. Excellent communication skills
7. Compensation: INR 321,000 per year
8. Joining date: -
9. Job location: Indore (M.P.), onsite position
10. Probation period: 90 days
